In [1]:
import yaml
import copy
import os

multi_task_config={'output_dir': '/n/home12/cfpark00/WM_1/data/experiments/pt1/evals/multi_task',
 'experiment_dir': '/n/home12/cfpark00/WM_1/data/experiments/pt1',
 'dataset_path': '/n/home12/cfpark00/WM_1/data/datasets/multitask_pt1',
 'cities_csv': '/n/home12/cfpark00/WM_1/data/datasets/cities/cities.csv',
 'eval_batch_size': 512,
 'max_generation_length': 256,
 'temperature': 0.0,
 'top_k': 1,
 'do_sample': False,
 'checkpoints': 'all',
 'device': 'cuda',
 'seed': 42,
 'save_full_results': True,
 'plot_log_scale': False}

task_config={'output_dir': 'data/experiments/pt1_ft2-4/evals/atlantis_compass',
 'experiment_dir': 'data/experiments/pt1_ft2-4',
 'dataset_path': 'data/datasets/compass_100k_atlantis_required',
 'eval_batch_size': 512,
 'max_generation_length': 256,
 'temperature': 0.0,
 'top_k': 1,
 'do_sample': False,
 'checkpoints': 'all',
 'device': 'cuda',
 'seed': 42,
 'save_full_results': True,
 'plot_log_scale': False}

In [2]:
all_yaml_paths={}
for exp_name in ["pt1",*["pt1_ft1-"+str(i) for i in range(1,8)],*["pt1_ft2-"+str(i) for i in range(1,22)],*["pt1_ft3-"+str(i) for i in range(1,8)],*["pt1_ftwb1-"+str(i) for i in range(1,8)],*["pt1_ftwb2-"+str(i) for i in range(1,22)],*["pt1_ftwb3-"+str(i) for i in range(1,8)]]:
    fol = f"./{exp_name}"
    os.makedirs(fol, exist_ok=True)
    
    yaml_paths=[]
    config = copy.deepcopy(multi_task_config)
    config["output_dir"] = f'/n/home12/cfpark00/WM_1/data/experiments/{exp_name}/evals/multi_task'
    config["experiment_dir"] = f'data/experiments/{exp_name}'
    yaml_path=os.path.join(fol, "multi_task.yaml")
    with open(yaml_path, "w") as f:
        yaml.dump(config, f)
    yaml_paths.append(yaml_path)
    
    for task in ["distance","trianglearea","angle","compass","inside","perimeter","crossing"]:
        for atlantis in [False,True]:
            taskname=("atlantis_"+task) if atlantis else task
            config=copy.deepcopy(task_config)
            config["output_dir"] = f'/n/home12/cfpark00/WM_1/data/experiments/{exp_name}/evals/{taskname}'
            config["experiment_dir"] = f'data/experiments/{exp_name}'
            if atlantis:
                config["dataset_path"]=f'data/datasets/{task}_100k_atlantis_required'
            else:
                config["dataset_path"]=f'data/datasets/{task}_1M_no_atlantis'
            yaml_path=os.path.join(fol, f"{taskname}.yaml")
            with open(yaml_path, "w") as f:
                yaml.dump(config, f)
            yaml_paths.append(yaml_path)
    
    bashfilebase="/n/home12/cfpark00/WM_1/scripts/eval/ftset"
    bashfilepath=os.path.join(bashfilebase,"eval_"+exp_name+".sh")
    with open(bashfilepath,"w") as f:
        for yaml_path in yaml_paths:
            f.write(f"uv run python src/eval/evaluate_checkpoints.py {os.path.abspath(yaml_path)} --overwrite\n")

In [3]:
all_yaml_paths={}
for n_t in [2]:#[1,2,3]:
    for n_i in range(1,8):#range(1,22):#[1,2,3,4,5,6,7]:
        exp_name = f"pt1_ftwb{n_t}-{n_i}"
        fol = f"./{exp_name}"
        os.makedirs(fol, exist_ok=True)
        
        yaml_paths=[]
        config = copy.deepcopy(multi_task_config)
        config["output_dir"] = f'/n/home12/cfpark00/WM_1/data/experiments/{exp_name}/evals/multi_task'
        config["experiment_dir"] = f'data/experiments/{exp_name}'
        yaml_path=os.path.join(fol, "multi_task.yaml")
        with open(yaml_path, "w") as f:
            yaml.dump(config, f)
        yaml_paths.append(yaml_path)
        
        for task in ["distance","trianglearea","angle","compass","inside","perimeter","crossing"]:
            for atlantis in [False,True]:
                taskname=("atlantis_"+task) if atlantis else task
                config=copy.deepcopy(task_config)
                config["output_dir"] = f'/n/home12/cfpark00/WM_1/data/experiments/{exp_name}/evals/{taskname}'
                config["experiment_dir"] = f'data/experiments/{exp_name}'
                if atlantis:
                    config["dataset_path"]=f'data/datasets/{task}_100k_atlantis_required'
                else:
                    config["dataset_path"]=f'data/datasets/{task}_1M_no_atlantis'
                yaml_path=os.path.join(fol, f"{taskname}.yaml")
                with open(yaml_path, "w") as f:
                    yaml.dump(config, f)
                yaml_paths.append(yaml_path)
        
        bashfilebase="/n/home12/cfpark00/WM_1/scripts/eval/ftset"
        bashfilepath=os.path.join(bashfilebase,"eval_"+exp_name+".sh")
        with open(bashfilepath,"w") as f:
            for yaml_path in yaml_paths:
                f.write(f"uv run python src/eval/evaluate_checkpoints.py {os.path.abspath(yaml_path)} --overwrite\n")